In [323]:
# import h5 reader
import h5py
import numpy as np
import pandas as pd
import time
from os import listdir
from os.path import isfile, join
import pickle
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib
import scipy.optimize as opt
from matplotlib.ticker import LinearLocator
from scipy.interpolate import griddata, interp1d
import emcee
import joblibs
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

work_path = "/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/2d/InverseExp/"
h5_path = work_path + "output/fault/"

# Gaussian predictor

In [324]:
# Import Gaussian-regression related functions
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# Pre-process the data
from sklearn import preprocessing

# function train_GP
class GP_predictor:
    # Constructor
    def __init__(self, 
                 input_set, 
                 observation_set, 
                 GPkernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-3, 1e3)), 
                 n_optimizers = 9):
        # Scale input data
        self.input_set = [list(x) for x in input_set]
        self.observation_set = [list(x) for x in observation_set]
        self.input_dimension = len(self.input_set[0])
        self.observation_dimension = len(self.observation_set[0])
        self.trainset_length = len(self.input_set)
        
        self.input_scaler = preprocessing.StandardScaler()
        self.input_scaler.fit(np.array(self.input_set))
        
        # Scale output data
        self.observation_scaler = preprocessing.StandardScaler()
        self.observation_scaler.fit(np.array(observation_set))
        
        # Fit Gaussian process
        self.GP = GaussianProcessRegressor(kernel = GPkernel, n_restarts_optimizer = n_optimizers)
        # self.GP = MyGPR(kernel = GPkernel, n_restarts_optimizer = n_optimizers, max_iter = max_iterations)
        self.GP.fit(self.input_scaler.transform(np.array(self.input_set)), 
                    self.observation_scaler.transform(np.array(self.observation_set)))
        
    # Predict on a new input set
    def predict(self, new_input_set):
        # Predict new observation
        new_observation = self.observation_scaler.inverse_transform(
            self.GP.predict(
                self.input_scaler.transform(np.array(list(new_input_set)).reshape([-1, self.input_dimension]))
            )
        )
        
        return new_observation
        
    

In [378]:
np.array_split(a, 3)

[array([[1., 2.],
        [3., 1.],
        [4., 5.]]),
 array([[6., 7.],
        [9., 0.],
        [4., 5.]]),
 array([[6., 7.],
        [9., 0.]])]

In [359]:
# Class RunABatch
class RunABatch:
    # Constructor
    def __init__(self, input_set, work_path, FourierTerms = 16, distanceAbove = 2e-3, 
                 nOfQueryPts = 10, obsFlag = 'fault', fourierFlag = False):
        # Initialize data paths and batch parameters
        self.work_path = work_path
        self.input_set = [tuple(i) for i in input_set]
        self.h5_path = work_path + "output/faultFiles/"
        self.frontsurf_path = work_path + "output/frontsurfFiles/"
        self.FourierTerms = FourierTerms
        self.distanceAbove = distanceAbove
        self.nOfQueryPts = nOfQueryPts
        self.fourierFlag = fourierFlag
        
        # Store all cases in the h5_path
        self.existingCasesFile = self.h5_path + "CaseList.csv"
        
        # Flag for whether cases have been run
        self.casesExcuted = False
        
        # Get existing cases
        self.getExistingCasesOfInputSet()
        
        # Get input_set_toRun
        self.input_set_toRun = list(set(self.input_set) - self.existingCases)
        
        # Run cases
        self.runCases(self.input_set_toRun)
        
        # Get obsevations
        # self.Observations = self.getObservations(self.input_set)
        if obsFlag == 'fault':
            self.Observations = self.getObservations(self.input_set, self.nOfQueryPts)
        elif obsFlag == 'front':
            self.Observations = self.getObservationsFrontSurf(self.input_set, self.nOfQueryPts, self.distanceAbove)
        else:
            self.Observations = self.getObservationsEveryWhere(self.input_set)
            
    # Inline function gets [A, B] list
    def getABfwVw(self, fileName):
        A_idx = fileName.find('A')
        B_idx = fileName.find('B')
        fw_idx = fileName.find('f')
        Vw_idx = fileName.find('V')
        slash_idx = fileName.find('-')
        
        # Change this part !! Before applying to new name convention
        A = float(fileName[A_idx + 1 : B_idx - 1])
        B = float(fileName[B_idx + 1 : fw_idx - 1])
        fw = float(fileName[fw_idx + 2 : Vw_idx - 1])
        Vw = float(fileName[Vw_idx + 2 : slash_idx])
        
        return (A, B, fw, Vw)
    
    
    # Function get_existing_files_set
    def getExistingCasesOfInputSet(self):
        # Get all .h5 file names as a list
        myPath = self.h5_path
        onlyFiles = [f for f in listdir(myPath) if (isfile(join(myPath, f)) and f[-8 : ] == 'fault.h5')]
        self.existingCases = set([self.getABfwVw(f) for f in onlyFiles])
        
    # Function runCases
    def runCases(self, input_set):
        shell_pathR = self.work_path + "RunJobsJP.sh"
        shell_pathW = self.work_path + "RunJobsJP-" + str(threadNo) + ".sh"
        shellRead = open(shell_pathR, 'r')
        list_of_lines = shellRead.readlines()
        shellRead.close()

        AA = [ele[0] for ele in input_set]
        BB = [ele[1] for ele in input_set]
        DD = [ele[2] for ele in input_set]
        TT = [ele[3] for ele in input_set]
        FF = [ele[4] for ele in input_set]
        
        list_of_lines[9] = "AA=" + str(tuple(AA)).replace(',', '') + "\n"
        list_of_lines[10] = "BB=" + str(tuple(BB)).replace(',', '') + "\n"
        list_of_lines[11] = "DD=" + str(tuple(DD)).replace(',', '') + "\n"
        list_of_lines[12] = "TT=" + str(tuple(TT)).replace(',', '') + "\n"
        list_of_lines[13] = "FF=" + str(tuple(FF)).replace(',', '') + "\n"
        list_of_lines[14] = "threadNo=" + str(threadNo) + "\n"

        shellWrite = open(shell_pathW, 'w')
        shellWrite.writelines(list_of_lines)
        shellWrite.close()

        # Run the cases
        !source $shell_pathW
        
        self.casesExcuted = True
        
        # Return from shell
        return
    
    
    # Function getObservations for the input_set
    def getObservations(self, input_set, nOfQueryPts):
        # Initialize Observations
        Observations = []
        
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Rotation matrix Q
        alpha = 29 / 180 * np.pi
        Q = np.array([[np.cos(alpha), np.sin(alpha)], [-np.sin(alpha), np.cos(alpha)]])
        
        # VS start
        VSstart = np.array([0.006354, 0.003522])
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.h5_path + "A" + str(input_ele[0]) + "_B" + str(input_ele[1]) \
                      + "_fw" + str(input_ele[2]) + "_Vw" + str(input_ele[3]) + "-fault.h5"
                     
            f = h5py.File(h5_file, 'r')

            # Get time
            times = np.array(f['time']).reshape([-1])
            times = times - np.min(times)
            nOfTSteps = times.shape[0]
            
            # Get coordinates
            coords = np.array(f['geometry']['vertices']) - VSstart
            Qcoords = coords @ Q.transpose()
            XXs = Qcoords[:, 0]
            
            # Get Slip rates [time, nOfNodes, spaceDim]
            SlipRates = np.array(f['vertex_fields']['slip_rate'])
            SlipRatesMag = np.linalg.norm(SlipRates, ord = 2, axis = 2)
            nOfNodes = SlipRatesMag.shape[1]
            
            xqs = 1e-3 * np.linspace(0., 45., nOfQueryPts)
            
            # Store the slip rates
            slip_rate_func = interp1d(XXs, SlipRatesMag, kind = 'cubic')  # [nOfTSteps, nOfNodes]
            
            # Change slip rate x to [nOfNodes, nOfTSteps]
            slip_rate_x = slip_rate_func(xqs).transpose()
            
#             # DEBUG LINES
#             print('nOfTSteps: ', nOfTSteps)
#             print('nOfNodes: ', nOfNodes)
#             print('SlipRatesMag shape: ', SlipRatesMag.shape)
#             print('slip_rate_x shape: ', slip_rate_x.shape)
            
            # Get the observations
            if self.fourierFlag:
                # Find the Fourier coefficients
                FourierTerms = self.FourierTerms
                T = np.max(times)

                # Compute the Fourier terms
                Ks = np.array(range(FourierTerms))
                coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * times)
                VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in slip_rate_x], 0)

                # Compute the fourier coefficients
                # print('time.shape: ', time.shape)
                observation = np.trapz(VxcoskPiTt, x=times)
                # print("observation shape: ", observation.shape)
                # Append the result from this file
                Observations.append(observation)
            else:
                # Find nFourier terms of values, evenly spaced in [0, T]
                T = np.max(times)
                tts = np.linspace(0., T, self.FourierTerms)
                observation_func = interp1d(times, slip_rate_x)
                observation = observation_func(tts).reshape([-1])
                Observations.append(observation)
                
        Observations = np.array(Observations)
        return Observations
    
    # Function getObservationsFrontSurf for the input_set
    def getObservationsFrontSurf(self, input_set, nOfQueryPts, distanceAbove):
        # Initialize Observations
        Observations = []
        
        # Rotation matrix Q
        alpha = 29 / 180 * np.pi
        Q = np.array([[np.cos(alpha), np.sin(alpha)], [-np.sin(alpha), np.cos(alpha)]])
        
        # VS start
        VSstart = np.array([0.006354, 0.003522])
        
        # Query points
        distance_above = distanceAbove  # Distance above the interface
        
        # Set x_up query points
        x_up = 1e-3 * np.linspace(0., 45., nOfQueryPts)
        QueryPts_up = np.stack([x_up, distance_above * np.ones(x_up.shape)], axis = 1)
        QueryPts_dn = np.stack([x_up, -distance_above * np.ones(x_up.shape)], axis = 1)
        
        # nOfNodes
        nOfNodes = len(x_up)
            
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.frontsurf_path + "A" + str(input_ele[0]) + "_B" + str(input_ele[1]) \
                      + "_fw" + str(input_ele[2]) + "_Vw" + str(input_ele[3]) + "-domain.h5"
                     
            f = h5py.File(h5_file, 'r')
            
            # Get time
            times = np.array(f['time']).reshape([-1])
            times = times - np.min(times)
            nOfTSteps = times.shape[0]

            # Get coordinates
            coords = np.array(f['geometry']['vertices']) - VSstart
            Qcoords = coords @ Q.transpose()
            
            # Store the slip rates
            slip_rate_x = np.zeros([nOfTSteps, nOfNodes])
            
            # Get Slip rates
            velocity = np.array(f['vertex_fields']['velocity'])
            Qvelocity = velocity @ Q.transpose()
            
            for i in range(nOfTSteps):
                slip_rate_x[i, :] = - griddata(Qcoords, velocity[i, :, 0], QueryPts_up, method = 'cubic') \
                              + griddata(Qcoords, velocity[i, :, 0], QueryPts_dn, method = 'cubic')
            slip_rate_x = slip_rate_x.transpose()
            
            # Get the observations
            if self.fourierFlag:
                # Find the Fourier coefficients
                FourierTerms = self.FourierTerms
                T = np.max(times)

                # Compute the Fourier terms
                Ks = np.array(range(FourierTerms))
                coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * times)
                VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in slip_rate_x], 0)

                # Compute the fourier coefficients
                # print('time.shape: ', time.shape)
                observation = np.trapz(VxcoskPiTt, x=times)
                # print("observation shape: ", observation.shape)
                # Append the result from this file
                Observations.append(observation)
            
            else:
                # Find nFourier terms of values, evenly spaced in [0, T]
                T = np.max(times)
                tts = np.linspace(0., T, self.FourierTerms)
                observation_func = interp1d(times, slip_rate_x)
                observation = observation_func(tts).reshape([-1])
                Observations.append(observation)
        
        Observations = np.array(Observations)
        return Observations
    
    # Function getObservationsFrontSurf for the input_set
    def getObservationsEveryWhere(self, input_set):
        # Initialize Observations
        Observations = []
        
        # Rotation matrix Q
        alpha = 29 / 180 * np.pi
        Q = np.array([[np.cos(alpha), np.sin(alpha)], [-np.sin(alpha), np.cos(alpha)]])
        
        # VS start
        VSstart = np.array([0.006354, 0.003522])
            
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.frontsurf_path + "A" + str(input_ele[0]) + "_B" + str(input_ele[1]) \
                      + "_fw" + str(input_ele[2]) + "_Vw" + str(input_ele[3]) + "-domain.h5"
                     
            f = h5py.File(h5_file, 'r')
            
            # Get time
            times = np.array(f['time']).reshape([-1])
            times = times - np.min(times)
            nOfTSteps = len(times)
            
            # Get coordinates
            coords = np.array(f['geometry']['vertices']) - VSstart
            Qcoords = coords @ Q.transpose()
            
            # Get the node list in the observable zone
            nodeList = np.all(
                np.stack(
                    [Qcoords[:, 0] >= 0., Qcoords[:, 0] <= 46.e-3, Qcoords[:, 1] >= -15.e-3, Qcoords[:, 1] <= 15.e-3]
                ), 
                axis = 0 
            )
            
            # Get Slip rates
            velocity = np.array(f['vertex_fields']['velocity'])[:, nodeList, :]
            Qvelocity = velocity @ Q.transpose()
            # print("Qvelocity shape: ", Qvelocity.shape)
            # Collapse all measurements
            slip_rate_x = Qvelocity.reshape([Qvelocity.shape[0], -1]).transpose()
            
            
            
            # Find nFourier terms of values, evenly spaced in [0, T]
            T = np.max(times)
            tts = np.linspace(0., T, len(times) // 5)
            observation_func = interp1d(times, slip_rate_x)
            observation = observation_func(tts).reshape([-1])
            Observations.append(observation)
        
        Observations = np.array(Observations)
        return Observations
    

# Class BayersianInv

In [360]:
# Define Bayersian Inv that solves a Bayersian inversion problem
class BayersianInv:
    # Constructor
    def __init__(self, u_low, u_high, u, y, work_path, FourierTerms = 16, 
                 atol = 1.0e-6, si_eta = 0.5, n_samples = 20, MCMCsteps = 1000, save_path = "./", 
                 distanceAbove = 2e-3, nOfQueryPts = 10, obsFlag = 'fault', fourierFlag = False):
        # Set X has to be compact in R^k
        self.u_low = u_low
        self.u_high = u_high
        
        # The true value of u, as well as the observation y to be inverted
        self.u = u
        self.y = y
        
        # Other parameters
        self.input_dim = len(u_low)
        self.output_dim = len(y)
        self.work_path = work_path
        self.FourierTerms = FourierTerms
        self.atol = atol
        self.si_eta = si_eta
        self.n_samples = n_samples
        self.MCMCsteps = MCMCsteps
        self.save_path = save_path
        
        # Keep record of distanceAbove and nOfQueryPts in the field of view
        self.distanceAbove = distanceAbove
        self.nOfQueryPts = nOfQueryPts
        
        # Get where the observations are from
        self.obsFlag = obsFlag
        self.fourierFlag = fourierFlag
        
        # Keep records of the sampled points and the corresponding observations after iteration
        self.U = np.empty([0, self.input_dim])
        self.O = np.empty([0, self.output_dim])
        self.iterations = 0
        
        # Keep record of eta, MaxMinLenRatio, GaussianProcess Emulator, average error on the sampled points, 
        # and empirical mean and stdv of the posterior after each iteration
        self.etas = []
        self.maxMinDistRatio = []
        self.GPs = []
        self.avg_errors = []
        self.mean = []
        self.stdv = []
        
        # Maximum likelihood points
        self.maxLikelihoodUs = []
        self.maxLikelihoodUsPropL2Error = []
        self.maxLikelihoodObsPropL2Error = []
        
    # Get the accumulative probability function
    def log_prob(self, u, y):
        # Apply hard constraints
        u_reshape = u.reshape([-1, self.input_dim])
        normal_idx = np.all(
            np.concatenate(
                [u_reshape >= self.u_low, u_reshape <= self.u_high], axis = 1
            ), 
            axis = 1
        )
       
        # First prior is uniform distribution
        res = np.ones(len(u_reshape))
        res[~normal_idx] = -np.inf
        
        # Add all posteriors in each iteration (since this is log of probability density)
        if (np.sum(normal_idx) > 0):
            for i in range(self.iterations):
                res[normal_idx] += -0.5 / self.etas[i] ** 2 * (
                    np.sum(
                        (y - self.GPs[i].predict(u_reshape[normal_idx])) ** 2, 
                        axis = 1
                    )
                )
        
        # Return the log_probability at the current iteration
        return res
    
    # Compute statistics: sample and calculate mean and stdv
    def compute_stats(self, n_samples, n_steps = 1_000):
        # Sample for more points to update the empirical statistics
        sampler = emcee.EnsembleSampler(n_samples, 
                                        self.input_dim, 
                                        self.log_prob, args=[y], 
                                        vectorize = True)

        # Initialize uniformly as the starting point
        p0 = np.random.uniform(size = [n_samples, self.input_dim]) * (self.u_high - self.u_low) + self.u_low

        # Get the result
        sampler.run_mcmc(p0, n_steps)
        samples = sampler.get_last_sample().coords
        self.mean.append(np.mean(samples, axis = 0))
        self.stdv.append(np.std(samples, axis = 0))
        
        # Get maximum likelihood estimate
        fun = lambda u: -self.log_prob(u, self.y)
        newCenter = opt.minimize(fun, x0 = self.mean[-1], 
                                 bounds = [(self.u_low[i], self.u_high[i]) for i in range(len(self.u_low))]
                                ).x
        
        self.maxLikelihoodUs.append(newCenter)
        
        self.maxLikelihoodUsPropL2Error.append(
            np.linalg.norm(self.u - self.maxLikelihoodUs[-1]) 
            / np.linalg.norm(self.u)
        )
        self.maxLikelihoodObsPropL2Error.append(
            np.linalg.norm(self.y - self.GPs[-1].predict(self.maxLikelihoodUs[-1])) 
            / np.linalg.norm(self.y)
        )
        
    # Get average error of a batch
    def get_avg_error_of_a_batch(self, myBatch):
        return np.mean(np.sum((myBatch.Observations - self.y) ** 2, axis = 1))
    
    # Run one iteration
    def runOneIteration(self, n_samples, n_stat_samples):
        # Sample from the current distribution for u and get the current observations
        sampler = emcee.EnsembleSampler(n_samples, 
                                        self.input_dim, 
                                        self.log_prob, args=[y], 
                                        vectorize = True)
        
        # Initialize uniformly as the starting point
        p0 = np.random.uniform(size = [n_samples, self.input_dim]) * (self.u_high - self.u_low) + self.u_low
        
        # Get the result
        sampler.run_mcmc(p0, self.MCMCsteps)
        self.samples = sampler.get_last_sample().coords
        
        # If there is self_etas, this is to make sure MaxMinDistRatio does not go too large
        if len(self.etas) > 0:
            eta_times = 0
            while (eta_times < 2) and (MaxMinDistRatio(self.samples) > 100.):
                # Double the last eta
                self.etas[-1] = 2. * self.etas[-1]
                eta_times += 1
                
                # Re-sample
                sampler = emcee.EnsembleSampler(n_samples, 
                                                self.input_dim, 
                                                self.log_prob, args=[y], 
                                                vectorize = True)

                # Initialize uniformly as the starting point
                p0 = np.random.uniform(size = [n_samples, self.input_dim]) * (self.u_high - self.u_low) + self.u_low

                # Get the result
                sampler.run_mcmc(p0, self.MCMCsteps)
                self.samples = sampler.get_last_sample().coords
                MCsteps = 0
                while (MCsteps < self.MCMCsteps / 10) and (MaxMinDistRatio(self.samples) > 10.):
                    sampler.run_mcmc(sampler.get_last_sample(), 1)
                    self.samples = sampler.get_last_sample().coords
                    MCsteps += 1
            
            # print("eta_times, MaxMinDistRatio: ", eta_times, MaxMinDistRatio(self.samples))


        # Run the forward map with the samples
        myBatch = RunABatch(self.samples, self.work_path, FourierTerms = self.FourierTerms, 
                            distanceAbove = self.distanceAbove, nOfQueryPts = self.nOfQueryPts, 
                            obsFlag = self.obsFlag, fourierFlag = self.fourierFlag)
        self.U = np.append(self.U, self.samples, axis = 0)
        self.O = np.append(self.O, myBatch.Observations, axis = 0)

        # Get a new Gaussian process
        myGP = GP_predictor(self.samples, myBatch.Observations)

        # Update the recorded variables of the process
        self.GPs.append(myGP)
        self.iterations += 1
        self.etas.append(self.si_eta)
        self.avg_errors.append(self.get_avg_error_of_a_batch(myBatch))
        self.maxMinDistRatio.append(MaxMinDistRatio(self.samples))
        
        # Update stats
        self.compute_stats(n_stat_samples)
        
        # Converging flag False
        return False
    
    # Run function
    def run(self, n_iter_max = 10, n_stat_samples = 1000):
        for i in range(n_iter_max):
            # In each iteration
            print("="*30, " Iteration ", str(self.iterations), " ", "="*30)
            
            # Run one iteration
            start_time = time.time()
            converged = self.runOneIteration(self.n_samples, n_stat_samples)
            end_time = time.time()
            print("Time cost: ", str(end_time - start_time), " s")
            # print("Average error in this iteration: ", self.avg_errors[-1])
            print("Maximum to minimum distance in the sample points: ", self.maxMinDistRatio[-1])
            # print("Mean of the posterior after this iteration: ", self.mean[-1])
            # print("Stdv of the posterior after this iteration: ", self.stdv[-1])
            print("Max likelihood estimate of the posterior after this iteration: ", self.maxLikelihoodUs[-1])
            print("L2 error of u: ", self.maxLikelihoodUsPropL2Error[-1])
            print("L2 error of y: ", self.maxLikelihoodObsPropL2Error[-1])
        # Save the current model to files
        self.save()
        
    # Save the model to files
    def save(self):
        savePath = self.save_path + "models/A{0}_B{1}_fw{2}_Vw{3}_dist{4}_nOfQPts{5}_eta{6}-{7}-Fourier-{8}.pickle".format(self.u[0], self.u[1], self.u[2], self.u[3], self.distanceAbove, self.nOfQueryPts, self.si_eta, self.obsFlag, self.fourierFlag)
        with open(savePath, 'wb') as file:
            pickle.dump(self, file)
    
    # Plot the regression process into a gif file
    def plotGIF(self):
        plotSeries(self, self.u, self.y, self.save_path)

        
# ===================================== Other Auxiliary functions ==========================================
# Function MaxMinDistRatio, compute the ratio of maximum to minimum distance among pts, 
# for the gaussian process to converge with a reasonable length-scaled kernel, 
# this value should not be too large
def MaxMinDistRatio(pts):
    matrix = [[np.linalg.norm(pts[i] - pts[j]) for j in range(i + 1, len(pts))] for i in range(len(pts))]
    vec = [x for y in matrix for x in y]
    return max(vec) / min(vec)


# Calculate the corresponding (to y) log likelihood of iter_step on u
def log_prob_best(self, iter_step, u, y):
    # Apply hard constraints
    u_reshape = u.reshape([-1, self.input_dim])
    normal_idx = np.all(np.concatenate([u_reshape >= self.u_low, u_reshape <= self.u_high], axis = 1), axis = 1)

    # First prior is uniform distribution
    res = np.ones(len(u_reshape))
    res[~normal_idx] = -np.inf

    # Add all posteriors in each iteration
    if (np.sum(normal_idx) > 0):
        for i in range(iter_step + 1):
            res[normal_idx] += -0.5 / self.etas[i] ** 2 * (
                np.sum(
                    (y - self.GPs[i].predict(u_reshape[normal_idx])) ** 2, 
                    axis = 1
                )
            )

    # Return the log_probability at the current iteration
    return res

# Plot the given regression process on (analytical_u, analytical_y) and save the GIF
def plotSeries(self, analytical_u, analytical_y, save_path, dpi_value = 300):
    # Calculate U_plot
    minU = self.u_low
    maxU = self.u_high
    nOfGridPoints = 100

    xis = []
    for i in range(minU.shape[0]):
        xis.append(np.linspace(minU[i], maxU[i], nOfGridPoints))

    # Generate grid and draw predictions
    UPlotGrid = np.meshgrid(xis[0], xis[1])
    UPlotGrid = np.stack(UPlotGrid, axis = 2)
    UPlotGridFat = UPlotGrid.reshape([nOfGridPoints * nOfGridPoints, maxU.shape[0]])
    
    gifName = save_path + "figures/A{0}_B{1}.gif".format(analytical_u[0], analytical_u[1])
    
    writer = imageio.get_writer(gifName, mode='I', duration = 1.0)
    
    # Maximum likelihood points
    self.maxLikelihoodUs = []
    self.maxLikelihoodUsPropL2Error = []
    self.maxLikelihoodObsPropL2Error = []
    
    # Plot the series
    for i in range(0, self.iterations):
        plt.figure(figsize = (7, 6), dpi = dpi_value)
        YPlotGridFat = log_prob_best(self, i, UPlotGridFat, analytical_y)
        YPlotGridFat = YPlotGridFat - np.max(YPlotGridFat)
        
        # Get optimized u
        idx = np.argmax(YPlotGridFat)
        
        # self.maxLikelihoodUs.append(UPlotGridFat[idx])
        
        fun = lambda u: -log_prob_best(self, i, u, analytical_y)
        newCenter = opt.minimize(fun, x0 = UPlotGridFat[idx], 
                                 bounds = [(self.u_low[i], self.u_high[i]) for i in range(len(self.u_low))]
                                ).x
        
        # DEBUG LINES
        # print("=" * 60)
        # print("i, u = ", i, newCenter)
        
        # print("-fun(u_opt), log_prob_best(u_opt): ", -fun(newCenter), log_prob_best(self, i, newCenter, analytical_y))
        # print("-fun(grid_opt), log_prob_best(grid_opt): ", -fun(UPlotGridFat[idx]), log_prob_best(self, i, UPlotGridFat[idx], analytical_y))
        self.maxLikelihoodUs.append(newCenter)
        
        self.maxLikelihoodUsPropL2Error.append(
            np.linalg.norm(analytical_u - self.maxLikelihoodUs[-1]) 
            / np.linalg.norm(analytical_u)
        )
        self.maxLikelihoodObsPropL2Error.append(
            np.linalg.norm(analytical_y - self.GPs[i].predict(self.maxLikelihoodUs[-1])) 
            / np.linalg.norm(analytical_y)
        )
        
        YPlotGrid = YPlotGridFat.reshape([nOfGridPoints, nOfGridPoints])
        cp = plt.contourf(UPlotGrid[:, :, 0], UPlotGrid[:, :, 1], np.maximum(YPlotGrid, -5.))
        
        # Give the color bar
        cbar = plt.colorbar(cp)
        plt.clim([-5., 0.])
        cbar.set_label('$-\\Phi(u)$', fontsize = 20)
        
        # Scatter the sample points
        plt.scatter(self.U[ :(i + 1) * self.n_samples, 0], 
                    self.U[ :(i + 1) * self.n_samples, 1], s = 1, color = 'white')
        plt.scatter(analytical_u[0], analytical_u[1], s = 15, color = 'red')
        plt.xlabel('$u_1$', fontsize = 20)
        plt.ylabel('$u_2$', fontsize = 20)
        plt.title("The " + str(i) + " th iteration")
        figName = save_path + "figures/shit" + str(i) + ".png"
        plt.savefig(figName, dpi = dpi_value)
        
        # Save the figures into a gif
        image = imageio.imread(figName)
        writer.append_data(image)
        !rm $figName
    writer.close()
    
    # Save a figure of L2PropError of u
    plt.figure(figsize = (7, 6), dpi = dpi_value)
    plt.scatter(range(1, 1 + len(self.maxLikelihoodUsPropL2Error)), self.maxLikelihoodUsPropL2Error, s = 20)
    plt.xlabel('Iteration', fontsize = 20)
    plt.ylabel('$\|u-u_{true}\|/\|u_{true}\|$', fontsize = 20)
    # plt.ylim([0., 0.01])
    plt.title('Relative L2 error of $u$', fontsize = 20)
    figName = save_path + "figures/A{0}_B{1}_propL2UError.png".format(analytical_u[0], analytical_u[1])
    plt.savefig(figName, dpi = dpi_value)

    # Save a figure of L2PropError of y
    plt.figure(figsize = (7, 6), dpi = dpi_value)
    plt.scatter(range(1, 1 + len(self.maxLikelihoodUsPropL2Error)), self.maxLikelihoodObsPropL2Error, s = 20)
    plt.xlabel('Iteration', fontsize = 20)
    plt.ylabel('$\|y-y_{true}\|/\|y_{true}\|$', fontsize = 20)
    # plt.ylim([0., 0.01])
    plt.title('Relative L2 error of $y$', fontsize = 20)
    figName = save_path + "figures/A{0}_B{1}_propL2YError.png".format(analytical_u[0], analytical_u[1])
    plt.savefig(figName, dpi = dpi_value)
    
    

In [361]:
# Load from file
def loadInvModel(model_path):
    with open(model_path, 'rb') as file:
        myInv = pickle.load(file)
    return myInv

# Test an inversion

In [362]:
# Test run a batch
input_set = [[0.016, 0.011, 0.1, 2], [0.016, 0.012, 0.2, 3]]
distanceAbove = 1e-3
nOfQueryPts = 10
fTerms = 16
obsFlag = 'everywhere'
fourierFlag = False
myBatch = RunABatch(input_set, work_path, fTerms, distanceAbove, nOfQueryPts, obsFlag, fourierFlag)

/home/shengduo/InverseProblems/GPRWorkingField


In [363]:
myBatch.Observations.shape

(2, 4446)

In [364]:
# Generate test batch
u_low = np.array([0.010, 0.005, 0.1, 1.])
u_high = np.array([0.02, 0.015, 0.5, 5.])
nSamples = 20
siEta = 1.0
print("Test input set: ", myBatch.input_set)
print("Work path: ", work_path)

Test input set:  [(0.016, 0.011, 0.1, 2), (0.016, 0.012, 0.2, 3)]
Work path:  /home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/2d/InverseExp/


In [367]:
# Run an inversion test:
testCase_idx = 0
u = myBatch.input_set[testCase_idx]
y = myBatch.Observations[testCase_idx]
myInv = BayersianInv(u_low, u_high, u, y, work_path, FourierTerms = fTerms, 
                     n_samples = nSamples, 
                     si_eta = siEta, 
                     nOfQueryPts = nOfQueryPts, 
                     distanceAbove = distanceAbove, 
                     obsFlag = obsFlag, 
                     fourierFlag = fourierFlag
                    )
myInv.run(n_iter_max = 10)

# # Save to file
# import pickle

# #save it
# with open(f'./models/myInvA4_B6.pickle', 'wb') as file:
#     pickle.dump(myInv, file) 

==============================  Iteration  0   ==============================
Running case A0.014367833162257121_B0.008258469936587288_fw0.1092103667193226_Vw2.542244762956047
Finished in 95 s!

Running case A0.012761701211224157_B0.005143853042074895_fw0.10367060701469659_Vw3.2753609041487564
Finished in 95 s!

Running case A0.011127283179622378_B0.005856818167481423_fw0.29559903776610064_Vw3.1287098692180666
Finished in 94 s!

Running case A0.017824994201156668_B0.00659026910872996_fw0.3975142374806985_Vw3.0118706129161517
Finished in 88 s!

Running case A0.01708848478042562_B0.011488787381042297_fw0.41358655715006626_Vw2.247033972979728
Finished in 88 s!

Running case A0.012062216672680183_B0.008694887671304429_fw0.3845167618294725_Vw2.4608884635958432
Finished in 89 s!

Running case A0.014696700525907469_B0.014184821213887146_fw0.1863186882895181_Vw3.4742606772979245
Finished in 88 s!

Running case A0.017579708893319834_B0.009256090810439684_fw0.18228393043174276_Vw3.73513281993522

Running case A1.4365765839381357e-02_B0.013730725090511181_fw0.12328440573309449_Vw2.240713131188283
Finished in 85 s!

Running case A0.014218386211558427_B0.011216414703897367_fw0.16027425501857237_Vw3.1953115115669948
Finished in 86 s!

Running case A0.014513709029730114_B0.011527962768439717_fw0.11969222129426088_Vw3.049826526083519
Finished in 87 s!

Running case A0.014754289237879343_B0.014848956392222522_fw0.11273238872513466_Vw2.492418442932231
Finished in 87 s!

Running case A0.01484067531441625_B0.008787113557052244_fw0.21178184280929024_Vw1.5912299613837844
Finished in 87 s!

Running case A0.01414372757413443_B0.013509208646516772_fw0.11372899927152635_Vw3.2893960846705763
Finished in 86 s!

Running case A0.014938984137821848_B0.010400976413062714_fw0.21293175287360003_Vw1.842597973704722
Finished in 87 s!

Running case A0.014695935079561295_B0.013103551790907506_fw0.1213498647549457_Vw2.513718919973595
Finished in 86 s!

Running case A0.015493248841006758_B0.0094662628000519

Running case A1.45756709525776e-02_B0.012742147173084943_fw0.12351736417561177_Vw2.5006653268271135
Finished in 85 s!

Running case A0.01397712562279943_B0.01290231323673655_fw0.12459408000296064_Vw2.8488016619704783
Finished in 87 s!

Running case A0.014756326066413377_B0.011841231095279379_fw0.1297305747521827_Vw2.546615960550629
Finished in 88 s!

Running case A0.014429986390686532_B0.013124207794771792_fw0.10870023380780167_Vw2.6373230066696114
Finished in 88 s!

Running case A0.014667218110614149_B0.01233773612589656_fw0.10590304475595913_Vw2.5552760669138115
Finished in 88 s!

Running case A0.014527371606399003_B0.012435149673055909_fw0.11861975855421504_Vw2.5509736714803313
Finished in 88 s!

Running case A0.01446923618053414_B0.012411147717591591_fw0.1400673314310385_Vw2.6246052971637477
Finished in 88 s!

Running case A0.01624739725941416_B0.009917370288620485_fw0.2219192507543048_Vw1.49632080635316
Finished in 89 s!

Running case A0.014667252278674365_B0.012861620297988626_fw

Running case A1.9106929240749672e-02_B0.013996060169638207_fw0.4709241936513625_Vw1.5385371679569346
Finished in 86 s!

Running case A0.014665670554938856_B0.012251563866540046_fw0.11653431878874575_Vw2.448967131168602
Finished in 88 s!

Running case A0.014578236445642661_B0.012449230526376653_fw0.12153626269848174_Vw2.5233649661036592
Finished in 89 s!

Running case A0.014644292334333182_B0.01249318420980847_fw0.10429479736029285_Vw2.606616434202402
Finished in 89 s!

Running case A0.016025273399729763_B0.008855533748581595_fw0.21137593572772817_Vw1.5701110729635082
Finished in 89 s!

Running case A0.014700533681105379_B0.012660737918365672_fw0.10491757778100662_Vw2.596476005505269
Finished in 89 s!

Running case A0.014566303087741013_B0.012650190002480875_fw0.12477137097660522_Vw2.5166481955606064
Finished in 89 s!

Running case A0.014472787085360522_B0.012174458167447515_fw0.13434993402936773_Vw2.4230217036606185
Finished in 88 s!

Running case A0.014704833198741685_B0.0123173458235

In [368]:
myInv.run(n_iter_max = 10)

==============================  Iteration  10   ==============================
Running case A1.4803918916629534e-02_B0.012034995706718296_fw0.11399273400466274_Vw2.635853488900826
Finished in 86 s!

Running case A0.014694003675172923_B0.01265879643633399_fw0.10396207347436329_Vw2.598795434792494
Finished in 86 s!

Running case A0.014656451007806139_B0.012484103277718827_fw0.1188991921956464_Vw2.6379017045894577
Finished in 88 s!

Running case A0.019719346232290123_B0.01193174848498216_fw0.48393061602975795_Vw4.171168127846677
Finished in 89 s!

Running case A0.01651343054835238_B0.00854852090770217_fw0.25991071930378723_Vw1.498758975198224
Finished in 94 s!

Running case A0.01875098575716673_B0.006093354002991165_fw0.48869213058213745_Vw4.839293153581833
Finished in 94 s!

Running case A0.014761529659404768_B0.012073116780596492_fw0.1153070080200809_Vw2.3588662109479537
Finished in 94 s!

Running case A0.014716363203964033_B0.012293044373032927_fw0.11087721715788422_Vw2.607399060221104

Running case A1.4704523580777888e-02_B0.01238365381739001_fw0.10640906944219693_Vw2.6197943015303493
Finished in 87 s!

Running case A0.014594964477546659_B0.012919287812908416_fw0.10046239701201339_Vw2.5020689926778745
Finished in 88 s!

Running case A0.010188324252359854_B0.009967226573280493_fw0.3372960218583655_Vw3.594654891749399
Finished in 88 s!

Running case A0.017004902315908635_B0.014062695847410975_fw0.484221669023243_Vw4.182754971894142
Finished in 89 s!

Running case A0.014745828772427045_B0.012431934717015392_fw0.11859879277183191_Vw2.5553344206088857
Finished in 89 s!

Running case A0.014777848364219318_B0.012273949522043264_fw0.11160536459603475_Vw2.5858306939684446
Finished in 90 s!

Running case A0.014638949223449646_B0.012625461909612497_fw0.10491794611477535_Vw2.692960971538652
Finished in 89 s!

Running case A0.019305966711707742_B0.014560338132918875_fw0.49105253658422565_Vw3.314603298566703
Finished in 90 s!

Running case A0.015874550454281555_B0.0104606503128863

/home/shengduo/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-03. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Time cost:  2333.932382583618  s
Maximum to minimum distance in the sample points:  9.105931677353683e+02
Max likelihood estimate of the posterior after this iteration:  [0.01472011 0.01236731 0.11045672 2.52670466]
L2 error of u:  0.26306488298814434
L2 error of y:  0.2218884722257743
==============================  Iteration  14   ==============================
Running case A1.4589529602402541e-02_B0.012641594632083563_fw0.10473333597202024_Vw2.616751436368598
Finished in 84 s!

Running case A0.014673743276562744_B0.01255225108036398_fw0.10099245163768535_Vw2.647647704141789
Finished in 88 s!

Running case A0.01441989444744474_B0.012197302259515699_fw0.13114729579880352_Vw2.6651960359506544
Finished in 88 s!

Running case A0.016944835644086706_B0.00945475007013598_fw0.2570410600510311_Vw1.3559242981586588
Finished in 87 s!

Running case A0.01451436642589555_B0.012614509812405549_fw0.11721291342080475_Vw2.515381480550084
Finished in 88 s!

Running case A0.014804761356505053_B0.0123419

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/2d/InverseExp/output/frontsurfFiles/A0.014715792492967016_B0.012439331238589145_fw0.11248222017997507_Vw2.5036398003948466-domain.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [255]:
# Test observations
# Test run a batch
input_set = [[0.016, 0.011, 0.1, 2], [0.016, 0.012, 0.2, 3]]
distanceAbove = 1e-3
nOfQueryPts = 10
fTerms = 16
obsFlag = 'frontsurf'
myBatch = RunABatch(input_set, work_path, fTerms, distanceAbove, nOfQueryPts, obsFlag)

/home/shengduo/InverseProblems/GPRWorkingField


In [257]:
testCase_idx = 0
u = myBatch.input_set[testCase_idx]
y = myBatch.Observations[testCase_idx]
np.linalg.norm(y)

9.169259194698046e-06

In [280]:
# Test open an h5 file
nOfQueryPts = 10

# Rotation matrix Q
alpha = 29 / 180 * np.pi
Q = np.array([[np.cos(alpha), np.sin(alpha)], [-np.sin(alpha), np.cos(alpha)]])

# VS start
VSstart = np.array([0.006354, 0.003522])

# Query points
distance_above = distanceAbove  # Distance above the interface

# Set x_up query points
x_up = 1e-3 * np.linspace(0., 45., nOfQueryPts)
QueryPts_up = np.stack([x_up, distance_above * np.ones(x_up.shape)], axis = 1)
QueryPts_dn = np.stack([x_up, -distance_above * np.ones(x_up.shape)], axis = 1)

# nOfNodes
nOfNodes = len(x_up)
        

# Open the file
# h5_file = "/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/2d/InverseExp/output/frontsurfFiles/A0.016_B0.011_fw0.1_Vw2-domain.h5"
h5_file = "/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/2d/InverseExp/output/frontsurfFiles/A0.016_B0.012_fw0.2_Vw3-domain.h5"

f = h5py.File(h5_file, 'r')

# Get time
times = np.array(f['time']).reshape([-1])
times = times - np.min(times)
nOfTSteps = times.shape[0]
times = times / np.max(times)

# Get coordinates
coords = np.array(f['geometry']['vertices']) - VSstart
Qcoords = coords @ Q.transpose()

# Store the slip rates
slip_rate_x = np.zeros([nOfTSteps, nOfNodes])

# Get Slip rates
velocity = np.array(f['vertex_fields']['velocity'])
Qvelocity = velocity @ Q.transpose()

for i in range(nOfTSteps):
    slip_rate_x[i, :] = - griddata(Qcoords, velocity[i, :, 0], QueryPts_up, method = 'cubic') \
                  + griddata(Qcoords, velocity[i, :, 0], QueryPts_dn, method = 'cubic')
slip_rate_x = slip_rate_x.transpose()

# Find the Fourier coefficients
FourierTerms = 32
T = np.max(times)

# Compute the Fourier terms
Ks = np.array(range(FourierTerms))
coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * times)
VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in slip_rate_x], 0)

# Compute the fourier coefficients
# print('time.shape: ', time.shape)
observation = np.trapz(VxcoskPiTt, x=times)

In [281]:
np.max(slip_rate_x)

0.26449251741463475

In [282]:
times

array([0.        , 0.01001001, 0.02102102, 0.03203203, 0.04304304,
       0.05405405, 0.06506507, 0.07607608, 0.08608609, 0.0960961 ,
       0.10610611, 0.11611612, 0.12612613, 0.13613614, 0.14614615,
       0.15615616, 0.16716717, 0.17817818, 0.18918919, 0.2002002 ,
       0.21121121, 0.22222222, 0.23323323, 0.24424424, 0.25525526,
       0.26626627, 0.27727728, 0.28828829, 0.2992993 , 0.30930931,
       0.31931932, 0.32932933, 0.33933934, 0.34934935, 0.35935936,
       0.36936937, 0.37937938, 0.38938939, 0.3993994 , 0.40940941,
       0.41941942, 0.42942943, 0.43943944, 0.44944945, 0.45945946,
       0.46946947, 0.47947948, 0.48948949, 0.4994995 , 0.50950951,
       0.51951952, 0.52952953, 0.53953954, 0.54954955, 0.55955956,
       0.56956957, 0.57957958, 0.58958959, 0.5995996 , 0.60960961,
       0.61961962, 0.62962963, 0.63963964, 0.64964965, 0.65965966,
       0.66966967, 0.67967968, 0.68968969, 0.6996997 , 0.70970971,
       0.71971972, 0.72972973, 0.73973974, 0.74974975, 0.75975

In [283]:
max(observation)

0.044417879449190616

In [284]:
np.linalg.norm(observation)

0.09185036106914132

In [138]:
[u >= myInv.u_low, u <= myInv.u_high]

[array([ True,  True,  True,  True]), array([ True,  True,  True,  True])]